# Recogida de basuras por RL

Código principal para generar la simulación, añadir la red neuronal y el método de entrenamiento.

In [1]:
import math
import os
import sys
import time
import random

import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
import torch
import networkx as nx 
import osmnx as ox

Carga del grafo

In [2]:
# Obtenemos el grafo con los nodos de los contenedores de basura ya incluidos

G = ox.load_graphml("contenedores_resto_FCC_valencia_filtrado_grafo.graphml", node_dtypes={"osmid": str})
nodos, aristas = ox.graph_to_gdfs(G)

# Limitamos el rango de recogida de basuras al barrio de Benimàmet, València

nodos_zona = nodos[(-0.411672 > nodos["x"]) & (nodos["x"] > -0.431842) & (39.506424 > nodos["y"]) & (nodos["y"] > 39.495904)]
nodos_zona.to_csv("nodos_zona.csv", index=True, encoding="utf-8")

nodos_id = set(nodos_zona.index)
aristas_zona = aristas[aristas.index.get_level_values("u").isin(nodos_id) & aristas.index.get_level_values("v").isin(nodos_id)].copy()
aristas_zona.to_csv("aristas_zona.csv", index=True, encoding="utf-8")



In [3]:
# Separació Benimàmet en dos zonas (para entrenamiento)
norte = True
sud = False

if norte: 
    nodos_preparacion = nodos_zona[(-0.416603 > nodos["x"]) & (nodos["x"] > -0.431842) & (39.506424 > nodos["y"]) & (nodos["y"] > 39.498466)]
    nodos_preparacion = nodos_preparacion[(-0.423650 > nodos["x"]) | (39.500643 < nodos["y"])]
    nodos_preparacion = nodos_preparacion[(-0.422801 > nodos["x"]) | (39.501206 < nodos["y"])]
    nodos_preparacion = nodos_preparacion[(-0.421208 > nodos["x"]) | (39.501935 < nodos["y"])]
    nodos_zona = nodos_preparacion
    nodos_id = set(nodos_zona.index)
    aristas_zona = aristas[aristas.index.get_level_values("u").isin(nodos_id) & aristas.index.get_level_values("v").isin(nodos_id)].copy()

if sud:
    nodos_zona
    aristas_zona    

c:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\lib\site-packages\geopandas\geodataframe.py:1896: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
c:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\lib\site-packages\geopandas\geodataframe.py:1896: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
c:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\lib\site-packages\geopandas\geodataframe.py:1896: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
c:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\lib\site-packages\geopandas\geodataframe.py:1896: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


Preparación tuplas de nodos_zona y aristas_zona del entorno

In [4]:
# Se trabaja con diccionarios

def generacion_dicc_nodos(nodos_zona):
    nodos_dicc = {}
    for indice, fila in nodos_zona.iterrows():
        nodos_dicc[indice] = {
            "indice" : indice,
            "contenedor" : 1 if fila["contenedor"] == "True" else 0,  # 1 sí,  0 no
            "capacidad_contenedor" : 2 if fila["contenedor"] == "True" else 0, # 1 unidad de llenado sí, 0 unidades de llenado no
            "llenado" :  0.5 if fila["contenedor"] == "True" else 0,  # 0.5 (50%) (valor inicial aleatorio) sí,  0 (0%) no
            "posicion_camion" : 0,                                    # 1 sí,  0 no
            "llenado_camion" : 0                                      # Dato compartido en todos los nodos, normalizado a 1   
        }
    
    return nodos_dicc

def generacion_dicc_aristas(aristas_zona):
    aristas_dicc = {}
    for indice, fila in aristas_zona.iterrows():
        aristas_dicc[indice] = {
            "indice" : fila["osmid"], 
            "desde" : indice[0],
            "hasta" : indice[1],
            "distancia" : fila["length"],  # m
            "tiempo_recorrido" : fila["travel_time"], # s
            "velocidad_media" : fila["speed_kph"], # Km/h (velocidad indicada por graphml)
            "velocidad_max" : 30, # Km/h (Consideración inicial vias interubanas) 
            "via" : fila["highway"]
        }

    return aristas_dicc


nodos_dicc = generacion_dicc_nodos(nodos_zona)
aristas_dicc = generacion_dicc_aristas(aristas_zona)


Modificación indices de los nodos para trabajar despues con las GNN (se almacena la relación para mostrar resultados)

In [5]:
def modificacion_indices(nodos_dicc, aristas_dicc):
    
    nodos_dicc_str = {str(k): v for k, v in nodos_dicc.items()}

    mapeo = {osmid: idx for idx, osmid in enumerate(nodos_dicc_str.keys())}
    mapeo_inverso = {idx: osmid for osmid, idx in mapeo.items()}

    nodos_dicc_ind = {}
    for osmid, data in nodos_dicc_str.items():
        new_idx = mapeo[osmid]
        nodo_copy = data.copy()
        nodo_copy["indice"] = new_idx
        nodos_dicc_ind[new_idx] = nodo_copy

    aristas_dicc_ind = {}
    for osmid, data in aristas_dicc.items():
        u_idx = mapeo[str(data["desde"])]
        v_idx = mapeo[str(data["hasta"])]
        arista_copy = data.copy()
        arista_copy["desde"] = u_idx
        arista_copy["hasta"] = v_idx
        aristas_dicc_ind[osmid] = arista_copy

    return nodos_dicc_ind, aristas_dicc_ind, mapeo, mapeo_inverso

nodos_indice, aristas_indice, mapeo, mapeo_inverso = modificacion_indices(nodos_dicc, aristas_dicc)
print(mapeo_inverso[113])
print(nodos_indice)
print(aristas_indice)

contenedor_C/ CAMPAMENT, 87
{0: {'indice': 0, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 1: {'indice': 1, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 2: {'indice': 2, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 3: {'indice': 3, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 4: {'indice': 4, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 5: {'indice': 5, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 6: {'indice': 6, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 7: {'indice': 7, 'contenedor': 0, 'capacidad_contenedor': 0, 'llenado': 0, 'posicion_camion': 0, 'llenado_camion': 0}, 8: {'indice': 8, 'c

Preparación y entremaniento agente

In [6]:
from env_basuras_final import RecogidaBasurasEnv

# Asegúrate de tener estos dicts ya definidos en la sesión:
# nodos_indice, aristas_indice

env = RecogidaBasurasEnv(nodos_indice, aristas_indice, mascara=True, seed=22)



Función evaluación modelo

In [7]:
import numpy as np
import pandas as pd

def _base_env(env):
    # gymnasium / SB3: desciende por .env hasta llegar al entorno base
    e = env
    # Si existe .unwrapped úsalo; si no, cae al bucle .env
    if hasattr(e, "unwrapped"):
        return e.unwrapped
    while hasattr(e, "env"):
        e = e.env
    return e

def eval_episode(env, model, deterministic: bool = True, max_steps: int = 800):
    """
    Ejecuta 1 episodio y devuelve:
      - df_episode: DataFrame con (step, nodo_actual, contenedor, accion_tipo, accion_destino,
                                   accion_final, nodo_destino, reward, reward_acum)
      - ep_rew: recompensa total
      - steps: nº de pasos
    Lee atributos custom del entorno base (desenvuelto) para soportar Monitor/wrappers.
    """
    obs, _ = env.reset()
    ep_rew, steps = 0.0, 0
    traj = []

    while steps < max_steps:
        base = _base_env(env)  # <- entorno real con tus atributos
        nodo_origen = int(base.nodo_actual)
        contenedor = int(base.nodos_indice[nodo_origen]["contenedor"])

        act, _ = model.predict(obs, deterministic=deterministic)
        act = np.asarray(act).reshape(-1)
        tipo, destino = int(act[0]), int(act[1])

        # MUY IMPORTANTE: step al wrapper (Monitor) para que registre métricas
        obs, r, term, trunc, _ = env.step((tipo, destino))

        steps += 1
        ep_rew += float(r)

        # Vuelve a leer del base tras el step
        base = _base_env(env)
        nodo_destino = int(base.nodo_actual)

        traj.append({
            "step": steps,
            "nodo_actual": nodo_origen,
            "contenedor": contenedor,
            "accion_tipo": tipo,                         # 0 mover, 1 recoger
            "accion_destino": destino,
            "accion_final": "recoger" if tipo == 1 else "mover",
            "nodo_destino": nodo_destino,
            "reward": float(r),
            "reward_acum": float(ep_rew),
        })

        if term or trunc:
            break

    df_episode = pd.DataFrame(traj)
    print(f"[EVAL] determinista={deterministic} | recompensa={ep_rew:.2f} | steps={steps}")
    return df_episode, ep_rew, steps

# Entrenamiento A2C con paralelización

Ronda 10

In [8]:
# main.py
# ------------------------------------------------------------
# Lanza el entrenamiento con multiprocessing y guardado a CARPETAS.
# Asegúrate de definir/importar nodos_indice y aristas_indice ANTES.
# ------------------------------------------------------------
import os
from A2C_algoritmo import train_a2c

# Aquí debes tener ya:
# from tu_modulo_grafo import nodos_indice, aristas_indice
# o haberlos construido arriba en este mismo archivo/notebook.

ronda = "10"

def main():
    cfg = dict(
        total_timesteps = 12_000_000,
        n_envs = 8,
        n_steps = 64,            
        learning_rate = 3e-4,
        ent_coef_inicial = 2,
        ent_coef_final = 0.25,
        gamma = 0.995,            # Discount Reward
        beta_int = 0.8,           # Intrínseco  (necesario con la explícita?)
        alpha = 0.997,            # Intrínseco
        seed = 22,
        device = "cuda",
        run_name = f"ronda_{ronda}",
        models_dir = "./models/a2c",
        tb_dir = "./logs/tb-a2c",
        save_freq = 600_000,     # checkpoints cada 450k a carpeta
    )

    model, final_dir = train_a2c(
        nodos_indice = nodos_indice,
        aristas_indice = aristas_indice,
        **cfg
    )

    print("[FIN] Entrenamiento completado.")
    print("Carpeta final:", final_dir)

    return model


if __name__ == "__main__":
    model = main()

c:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\lib\site-packages\torch_geometric\typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: Could not find module 'C:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\Lib\site-packages\libpyg.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
c:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\lib\site-packages\torch_geometric\typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: [WinError 127] No se encontró el proceso especificado
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
c:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\lib\site-packages\torch_geometric\typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: [

[INFO] VecEnv usado: SubprocVecEnv | n_envs=8
Using cuda device
Logging to ./logs/tb-a2c\ronda_10\csv


c:\Users\jgilg\Desktop\Master_IA_VIU\TFM\codigo_github\rl_env\lib\site-packages\stable_baselines3\common\vec_env\vec_monitor.py:44: UserWarning: The environment is already wrapped with a `Monitor` wrapperbut you are wrapping it with a `VecMonitor` wrapper, the `Monitor` statistics will beoverwritten by the `VecMonitor` ones.
  warnings.warn(


-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 411       |
|    ep_rew_mean        | 33.3      |
| time/                 |           |
|    fps                | 497       |
|    iterations         | 100       |
|    time_elapsed       | 102       |
|    total_timesteps    | 51200     |
| train/                |           |
|    ent_coef           | 1.99      |
|    entropy_loss       | -0.251    |
|    explained_variance | -3.41e-05 |
|    learning_rate      | 0.0003    |
|    n_updates          | 99        |
|    policy_loss        | 0.0234    |
|    value_loss         | 4.66      |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 411       |
|    ep_rew_mean        | 24.5      |
| time/                 |           |
|    fps                | 494       |
|    iterations         | 200       |
|    time_elapsed       | 207       |
|    total_t

In [9]:
from stable_baselines3.common.monitor import Monitor

eval_env = Monitor(
    RecogidaBasurasEnv(
        nodos_indice = nodos_indice,
        aristas_indice = aristas_indice,
        steps_maximo = 800,
        mascara = True,
        seed = 123,
    )
)

for i in range(10):
    eval_episode(eval_env, model, deterministic=True, max_steps=800)

[EVAL] determinista=True | recompensa=-4.04 | steps=800
[EVAL] determinista=True | recompensa=-4.04 | steps=800
[EVAL] determinista=True | recompensa=-4.04 | steps=800
[EVAL] determinista=True | recompensa=-4.04 | steps=800
[EVAL] determinista=True | recompensa=-4.04 | steps=800
[EVAL] determinista=True | recompensa=-4.04 | steps=800
[EVAL] determinista=True | recompensa=-4.04 | steps=800
[EVAL] determinista=True | recompensa=-4.04 | steps=800
[EVAL] determinista=True | recompensa=-4.04 | steps=800
[EVAL] determinista=True | recompensa=-4.04 | steps=800


## Extra

Extracción pesos del modelo ya entrenado y guardado en carpeta

In [10]:
import torch
from stable_baselines3 import A2C
from sb3_policy_mascara import A2CPolicyGNNMasked

ronda_carga = "10"
# recrea el vec_env con los mismos spaces:
# (usa DummyVecEnv si solo vas a predecir/evaluar)
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from env_basuras_final import RecogidaBasurasEnv

def make_eval_env():
    return Monitor(RecogidaBasurasEnv(nodos_indice, aristas_indice, steps_maximo=800, mascara=True, seed=123))

vec_env = DummyVecEnv([make_eval_env])

# crea el modelo "vacío" (mismas policy_kwargs que en entrenamiento)
policy_kwargs = dict(hidden_dim=128, in_node_features=5, in_edge_features=2, n_tipos=2, max_nodes=len(nodos_indice), gnn_layers=3)
model_fold = A2C(policy=A2CPolicyGNNMasked, env=vec_env, policy_kwargs=policy_kwargs, device="cuda", verbose=0)

# carga pesos desde carpeta
folder = f"./models/a2c/ronda_{ronda_carga}/final"    # Varia dependiendo run
state_path = os.path.join(folder, "policy_state.pt")
opt_path   = os.path.join(folder, "optimizer_state.pt")

sd = torch.load(state_path, map_location="cuda")
model_fold.policy.load_state_dict(sd)

if os.path.exists(opt_path):
    opt_sd = torch.load(opt_path, map_location="cuda")
    if hasattr(model_fold.policy, "optimizer") and model_fold.policy.optimizer is not None:
        model_fold.policy.optimizer.load_state_dict(opt_sd)

print("Pesos cargados desde carpeta:", folder)

Pesos cargados desde carpeta: ./models/a2c/ronda_10/final


In [11]:
for i in range(10):
    eval_episode(env, model_fold, deterministic=False, max_steps=800)

[EVAL] determinista=False | recompensa=10.21 | steps=146
[EVAL] determinista=False | recompensa=16.35 | steps=707
[EVAL] determinista=False | recompensa=12.20 | steps=370
[EVAL] determinista=False | recompensa=6.48 | steps=88
[EVAL] determinista=False | recompensa=9.23 | steps=103
[EVAL] determinista=False | recompensa=13.08 | steps=757
[EVAL] determinista=False | recompensa=12.18 | steps=131
[EVAL] determinista=False | recompensa=11.04 | steps=800
[EVAL] determinista=False | recompensa=10.13 | steps=309
[EVAL] determinista=False | recompensa=17.34 | steps=213


In [14]:
import os
import folium
import pandas as pd
import numpy as np
from shapely.geometry import LineString

# ================== 1) Ejecutar episodio ==================
deterministico = True  # True -> determinista, False -> no determinista
df_episode, R, S = eval_episode(env, model_fold, deterministic=deterministico, max_steps=800)
print(df_episode)

# ================== 2) GDFs / mapeos ==================
gdf_nodes = nodos_zona.copy()
gdf_edges = aristas_zona.copy()

# Asegura columnas u,v
if "u" not in gdf_edges.columns or "v" not in gdf_edges.columns:
    if isinstance(gdf_edges.index, pd.MultiIndex):
        gdf_edges = gdf_edges.reset_index()

# ================== 3) Serie de estados / visitas por nodo ==================
if not df_episode.empty:
    start_node_internal = int(df_episode.loc[0, "nodo_actual"])
    estados_internal = [start_node_internal] + df_episode["nodo_destino"].astype(int).tolist()
else:
    estados_internal = []

def map_internal(i):
    i = int(i)
    return mapeo_inverso[i] if i in mapeo_inverso else None

estados_orig = [map_internal(i) for i in estados_internal]
estados_orig = [nid for nid in estados_orig if nid in gdf_nodes.index]

visitas_counts = pd.Series(estados_orig, dtype="object").value_counts() if len(estados_orig) else pd.Series([], dtype="int64")
gdf_nodes["visitas"] = gdf_nodes.index.map(visitas_counts).fillna(0).astype(int)

# ================== 4) Recogidas por contenedor ==================
rows_pick = df_episode[df_episode["accion_tipo"] == 1].copy()
if not rows_pick.empty:
    rows_pick["node_id"] = rows_pick["nodo_actual"].map(map_internal)
    rows_pick = rows_pick[rows_pick["node_id"].isin(gdf_nodes.index)]
    pickups_counts = rows_pick["node_id"].value_counts()
else:
    pickups_counts = pd.Series([], dtype="int64")
gdf_nodes["pickups"] = gdf_nodes.index.map(pickups_counts).fillna(0).astype(int)

# ================== 5) Conteo dirigidos (u->v) ==================
moves = df_episode[(df_episode["accion_tipo"] == 0) & (df_episode["nodo_destino"] != df_episode["nodo_actual"])].copy()
if not moves.empty:
    moves["u"] = moves["nodo_actual"].map(map_internal)
    moves["v"] = moves["nodo_destino"].map(map_internal)
    moves = moves[moves["u"].isin(gdf_nodes.index) & moves["v"].isin(gdf_nodes.index)]
    edge_counts = moves.groupby(["u","v"]).size().to_dict()
else:
    edge_counts = {}

# Conjunto de aristas presentes para detectar bidireccionalidad
edges_set = set(zip(gdf_edges["u"], gdf_edges["v"]))

# ================== 6) Construir mapa ==================
if len(estados_orig) > 0:
    c0 = (float(gdf_nodes.loc[estados_orig[0], "y"]), float(gdf_nodes.loc[estados_orig[0], "x"]))
else:
    c0 = (39.4699, -0.3763)
m = folium.Map(location=[c0[0], c0[1]], zoom_start=15)

# ---- 6.a) Aristas: color fijo; grosor fijo; tooltip según direccionalidad; badges por dirección ----
WEIGHT_USED = 3
WEIGHT_UNUSED = 2

def interp_point(geom: LineString, t: float):
    try:
        p = geom.interpolate(t, normalized=True)
        return float(p.y), float(p.x)
    except Exception:
        rp = geom.representative_point()
        return float(rp.y), float(rp.x)

for _, erow in gdf_edges.iterrows():
    geom = erow.get("geometry")
    if geom is None:
        continue

    u = erow["u"]; v = erow["v"]
    cnt_uv = int(edge_counts.get((u, v), 0))
    cnt_vu = int(edge_counts.get((v, u), 0))
    has_reverse = (v, u) in edges_set  # ¿existe la fila inversa v->u?

    used = (cnt_uv > 0) or (cnt_vu > 0 if has_reverse else False)
    color = "#0000cc" if used else "#000000"
    weight = WEIGHT_USED if used else WEIGHT_UNUSED
    opacity = 0.9 if used else 0.5

    # Tooltip según direccionalidad
    if has_reverse:
        tooltip_txt = f"Arista {u} ↔ {v} · u→v: {cnt_uv} · v→u: {cnt_vu}"
    else:
        tooltip_txt = f"Arista {u} → {v} · pasadas: {cnt_uv}"

    # Dibujar línea
    if geom.geom_type == "LineString":
        coords = [(lat, lon) for lon, lat in geom.coords]
        folium.PolyLine(coords, color=color, weight=weight, opacity=opacity, tooltip=tooltip_txt).add_to(m)
    elif geom.geom_type == "MultiLineString":
        for part in geom:
            coords = [(lat, lon) for lon, lat in part.coords]
            folium.PolyLine(coords, color=color, weight=weight, opacity=opacity, tooltip=tooltip_txt).add_to(m)

    # Badges:
    #   - Siempre badge u→v si cnt_uv>0 en t=0.35
    #   - Badge v→u:
    #        * si NO existe fila (v,u), lo ponemos aquí en t=0.65
    #        * si SÍ existe fila (v,u), se pintará cuando toque esa fila (evitamos duplicado)
    if cnt_uv > 0:
        yx = interp_point(geom, 0.35)
        folium.Marker(
            [yx[0], yx[1]],
            icon=folium.DivIcon(html=f'''
                <div style="
                    background:#0000cc; color:white; padding:1px 4px;
                    border-radius:10px; font-size:10px; border:1px solid #001a80;
                    box-shadow:0 0 2px rgba(0,0,0,0.3);">
                    {cnt_uv}
                </div>'''),
            tooltip=f"{u} → {v}: {cnt_uv} pasadas"
        ).add_to(m)

    if cnt_vu > 0 and not has_reverse:
        yx = interp_point(geom, 0.65)
        folium.Marker(
            [yx[0], yx[1]],
            icon=folium.DivIcon(html=f'''
                <div style="
                    background:#0000cc; color:white; padding:1px 4px;
                    border-radius:10px; font-size:10px; border:1px solid #001a80;
                    box-shadow:0 0 2px rgba(0,0,0,0.3);">
                    {cnt_vu}
                </div>'''),
            tooltip=f"{v} → {u}: {cnt_vu} pasadas"
        ).add_to(m)

# ---- 6.b) Nodos: contenedores verdes (visitas + recogidas), otros grises (solo visitas) ----
for nid, nrow in gdf_nodes.iterrows():
    lat, lon = float(nrow["y"]), float(nrow["x"])
    visitas = int(nrow.get("visitas", 0))
    pickups = int(nrow.get("pickups", 0))
    is_container = (nrow["contenedor"] == "True")

    if is_container:
        color = "#0b6623"     # verde
        tooltip = f"Contenedor {nid} · visitas: {visitas} · recogidas: {pickups}"
        radius = 5
    else:
        color = "#666666"     # gris
        tooltip = f"Nodo {nid} · visitas: {visitas}"
        radius = 5

    folium.CircleMarker(
        location=[lat, lon],
        radius=radius,
        color=color, weight=1,
        fill=True, fill_color=color, fill_opacity=0.9,
        tooltip=tooltip
    ).add_to(m)

# ================== 7) Guardar ==================
base_dir = f"./test/A2C/ronda_{ronda_carga}"
base_dir_no_det = base_dir + "/no_deterministico"
os.makedirs(base_dir, exist_ok=True)
os.makedirs(base_dir_no_det, exist_ok=True)

if deterministico:
    html_path = os.path.join(base_dir, f"mapa_ruta_contenedores_ronda_{ronda_carga}_det.html")
    csv_path  = os.path.join(base_dir, f"df_episode_ronda_{ronda_carga}_det.csv")
else:
    html_path = os.path.join(base_dir_no_det, f"mapa_ruta_contenedores_ronda_{ronda_carga}_no_det.html")
    csv_path  = os.path.join(base_dir_no_det, f"df_episode_ronda_{ronda_carga}_no_det.csv")

m.save(html_path)
df_episode.to_csv(csv_path, index=True)
print(f"[OK] Mapa guardado en: {html_path}")
print(f"[OK] df_episode guardado en: {csv_path}")



[EVAL] determinista=True | recompensa=-4.04 | steps=800
     step  nodo_actual  contenedor  accion_tipo  accion_destino accion_final  \
0       1           79           0            0               1        mover   
1       2            1           0            0              90        mover   
2       3           90           0            0               7        mover   
3       4            7           0            0              21        mover   
4       5           21           0            0               8        mover   
..    ...          ...         ...          ...             ...          ...   
795   796          111           1            0              13        mover   
796   797           13           0            0              14        mover   
797   798           14           0            0              15        mover   
798   799           15           0            0             111        mover   
799   800          111           1            0              13 

In [13]:
print(mapeo['575777700'])

79
